In [6]:
import pandas as pd
import numpy as np

In [24]:
MONTHS = {'jan': 1, 'feb': 2, 'mar': 3, 'apr': 4, 'may': 5, 'jun': 6, 'jul': 7, 'aug': 8, 'sep': 9, 'oct': 10, 'nov': 11, 'dec': 12}

mixed_type_columns = [3, 5, 11, 28, 362, 392, 397, 402, 407, 412, 417, 437, 442, 447, 452, 457, 472, 477, 482, 492, 497, 502, 507, 517, 522, 527, 532, 542, 547, 552, 557, 562, 567, 572, 577, 582, 587, 592, 597, 602, 607, 612, 617, 622, 627, 632, 637, 642, 647, 652, 655, 668]

dtype_dict = {col: str for col in mixed_type_columns}

openei = pd.read_csv('../data/kentucky_rand.csv', dtype=dtype_dict)

openei['sourceparent'] = openei['sourceparent'].fillna('')


In [41]:
def find_consecutive_ranges(lst):
    if not lst:
        return []

    ranges = []
    start = 0

    for i in range(1, len(lst)):
        if lst[i] != lst[start]:
            ranges.append((start, i - 1))
            start = i

    # Final check to include the last range if it ends at the end of the list
    if len(lst) - start > 1:
        ranges.append((start, len(lst) - 1))

    return ranges

In [51]:
data_dict = {
    'utility': [],
    'type': [],
    'assessed': [],
    'period': [],
    'basic_charge_limit (imperial)': [],
    'basic_charge_limit (metric)': [],
    'month_start': [],
    'month_end': [],
    'hour_start': [],
    'hour_end': [],
    'weekday_start': [],
    'weekday_end': [],
    'charge (imperial)': [],
    'charge (metric)': [],
    'units': [],
    'Notes': []
}

In [52]:
index_2 = 0

#for i in range(len(openei_dummy)):
for i in range(23,24):
    # customer flat rate (need to add code checking if rate is zero)
    data_dict['utility'].append('electricity')
    data_dict['type'].append('customer')
    data_dict['assessed'].append('')
    data_dict['period'].append('')
    data_dict['basic_charge_limit (imperial)'].append('')
    data_dict['basic_charge_limit (metric)'].append('')
    data_dict['month_start'].append('')
    data_dict['month_end'].append('')
    data_dict['hour_start'].append('')
    data_dict['hour_end'].append('')
    data_dict['weekday_start'].append('')
    data_dict['weekday_end'].append('')
    data_dict['charge (imperial)'].append(openei['fixedchargefirstmeter'][i])
    data_dict['charge (metric)'].append(openei['fixedchargefirstmeter'][i])
    data_dict['units'].append('$/month')
    data_dict['Notes'].append(openei['source'][i] + ('\t' + openei['sourceparent'][i] if openei['sourceparent'][i] != '' else ''))

    # processing the array for time intervals

    MONTH_ARRAY = ['flatDemandMonth_jan', 'flatDemandMonth_feb', 'flatDemandMonth_mar', 'flatDemandMonth_apr', 'flatDemandMonth_may', 'flatDemandMonth_jun', 'flatDemandMonth_jul', 'flatDemandMonth_aug', 'flatDemandMonth_sep', 'flatDemandMonth_oct', 'flatDemandMonth_nov', 'flatDemandMonth_dec']
    sched = {}
    for j in range(len(MONTH_ARRAY)):
        sched[j] = (openei[MONTH_ARRAY[j]][i])
    print(f"sched: {sched}")
    
    ranges = find_consecutive_ranges(list(sched.values()))
        
    print(f"ranges: {ranges}, ranges length: {len(ranges)}")
    if len(ranges) == 0:
        ranges = [(1, 12)]

    time_index = 0
    tier_index = 0
    charge_limit = 0

    while time_index < len(ranges):
        try:
            # tier_str first to catch ValueErrors from null values in the dataframe
            tier_str = 'flatdemandstructure/period'+ str(int(sched[ranges[time_index][0]])) + '/tier'+ str(tier_index)
            data_dict['month_start'].append(str(ranges[time_index][0]))
            data_dict['month_end'].append(str(ranges[time_index][1]))
            print(tier_str)
            data_dict['utility'].append('electricity')
            data_dict['type'].append('demand')
            data_dict['assessed'].append('')
            data_dict['period'].append('')
            data_dict['basic_charge_limit (imperial)'].append(charge_limit)
            data_dict['basic_charge_limit (metric)'].append(charge_limit)
            data_dict['hour_start'].append('')
            data_dict['hour_end'].append('')
            # Not the case for all structures
            data_dict['weekday_start'].append('0')
            data_dict['weekday_end'].append('6')
            rate = openei[tier_str + 'rate'][i]
            if not np.isnan(openei[tier_str + 'adj'][i]):
                rate += openei[tier_str + 'adj'][i]
            print(rate)
            # TODO: Access the rate through the index of the keys in dictionary 
            data_dict['charge (imperial)'].append(rate)
            data_dict['charge (metric)'].append(rate)
            data_dict['units'].append(openei['flatdemandunit'][i])
            data_dict['Notes'].append('')


            max_str = 'flatdemandstructure/period' + str(int(sched[ranges[time_index][0]])) + '/tier'+ str(tier_index) + 'max'
            print(openei[max_str][i])
            if not np.isnan(openei[max_str][i]):
                charge_limit = openei[max_str][i]
                tier_index += 1
                print("ONE")
            else:
                time_index += 1
                print("TWO")

        except (ValueError, KeyError):
            break


print(data_dict)
for key, value in data_dict.items():
    print(f"{key}: {len(value)}")
new_df = pd.DataFrame(data_dict)
print(new_df)

sched: {0: np.float64(0.0), 1: np.float64(0.0), 2: np.float64(0.0), 3: np.float64(0.0), 4: np.float64(0.0), 5: np.float64(0.0), 6: np.float64(0.0), 7: np.float64(0.0), 8: np.float64(0.0), 9: np.float64(0.0), 10: np.float64(0.0), 11: np.float64(0.0)}
ranges: [(0, 11)], ranges length: 1
flatdemandstructure/period0/tier0
18.38
1000.0
ONE
flatdemandstructure/period0/tier1
18.42
2500.0
ONE
flatdemandstructure/period0/tier2
18.89
nan
TWO
{'utility': ['electricity', 'electricity', 'electricity', 'electricity'], 'type': ['customer', 'demand', 'demand', 'demand'], 'assessed': ['', '', '', ''], 'period': ['', '', '', ''], 'basic_charge_limit (imperial)': ['', 0, np.float64(1000.0), np.float64(2500.0)], 'basic_charge_limit (metric)': ['', 0, np.float64(1000.0), np.float64(2500.0)], 'month_start': ['', '0', '0', '0'], 'month_end': ['', '11', '11', '11'], 'hour_start': ['', '', '', ''], 'hour_end': ['', '', '', ''], 'weekday_start': ['', '0', '0', '0'], 'weekday_end': ['', '6', '6', '6'], 'charge (

In [ ]:
#misc code (ignore)
for i in range(3,4):
    # First demand tariff
    start_elec = 31
    end_elec = 99
    num_columns = 4
    for j in range(start_elec, end_elec, num_columns):
        if openei.iloc[3, j+1] == openei['flatdemandstructure/period0/tier0rate'][3]:
            print('yes')
        else:
            print(f"{openei.iloc[i, j+1]} {openei['flatdemandstructure/period0/tier0rate'][i]}")
        data_dict['utility'].append('electricity')
        data_dict['type'].append('demand')
        data_dict['assessed'].append('')
        data_dict['period'].append('')
        data_dict['basic_charge_limit (imperial)'].append('')
        data_dict['basic_charge_limit (metric)'].append('')
        data_dict['month_start'].append('')
        data_dict['month_end'].append('')
        data_dict['hour_start'].append('')
        data_dict['hour_end'].append('')
        data_dict['weekday_start'].append('')
        data_dict['weekday_end'].append('')
        data_dict['charge (imperial)'].append(openei['flatdemandstructure/period0/tier0rate'][i] + openei['flatdemandstructure/period0/tier1adj'][i])
        data_dict['charge (metric)'].append(openei['flatdemandstructure/period0/tier0rate'][i]+ openei['flatdemandstructure/period0/tier1adj'][i])
        data_dict['units'].append(openei['flatdemandunit'][i])
        data_dict['Notes'].append('')
    
    #Successive tariffs batched in sets of 4
    #for j in range(32)

    data_dict['utility'].append('electricity')
    data_dict['type'].append('demand')
    data_dict['assessed'].append('')
    data_dict['period'].append('')
    data_dict['basic_charge_limit (imperial)'].append('')
    data_dict['basic_charge_limit (metric)'].append('')
    data_dict['month_start'].append('')
    data_dict['month_end'].append('')
    data_dict['hour_start'].append('')
    data_dict['hour_end'].append('')
    data_dict['weekday_start'].append('')
    data_dict['weekday_end'].append('')
    data_dict['charge (imperial)'].append(openei['flatdemandstructure/period0/tier0rate'][i])
    data_dict['charge (metric)'].append(openei['flatdemandstructure/period0/tier0rate'][i])
    data_dict['units'].append('$/kW')
    data_dict['Notes'].append('')